# TrainingServingSkew
* Moniitors whether the alphas generated for inference are identical 
    to the alphas generated during training

In [ ]:
username = ""
task = ""

In [ ]:
import NBInclude: @nbinclude
import Statistics: mean
@nbinclude("../InferenceAlphas/Alpha.ipynb");

In [ ]:
function read_training_alpha(alpha::String, split::String; content = nothing)
    uid = parse(Int, get_recommendee_username()) + 1
    if isnothing(content)
        if occursin("explicit", lowercase(alpha))
            @assert isnothing(content)
            content = "explicit"
        end
        if occursin("implicit", lowercase(alpha)) || alpha in [
            "all/UserAverage",
            "all/UserVariance",
            "all/SimpleAverage",
            "$task/MLE.Ensemble",
        ]
            @assert isnothing(content)
            content = "implicit"
        end
        if occursin("Ptw", alpha)
            @assert isnothing(content)
            content = "ptw"
        end
        @assert !isnothing(content) alpha
    end
    df = read_alpha(alpha, split, task, content)
    mask = df.user .== uid
    df =
        RatingsDataset(user = df.user[mask], item = df.item[mask], rating = df.rating[mask])
    df
end;

In [ ]:
function average_errors(alpha, split; content = nothing)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split; content = content)
    if length(training.item) == 0
        return 0
    end
    abs.(training.rating .- serving.rating[training.item])
end

function relative_errors(alpha, split; content = nothing)
    serving = read_recommendee_alpha(alpha, "all")
    training = read_training_alpha(alpha, split; content = content)
    if length(training.item) == 0
        return 0
    end
    ϵ = eps(Float64)
    abs.(abs.(training.rating) ./ (abs.(serving.rating[training.item]) .+ ϵ) .- 1)
end;

In [ ]:
function maximum_error(alpha, split; content = nothing)
    maximum(average_errors(alpha, split; content = content))
end

function average_abs_error(alpha, split; content = nothing)
    mean(average_errors(alpha, split; content = content))
end

function average_rel_error(alpha, split; content = nothing)
    mean(relative_errors(alpha, split; content = content))
end;

In [ ]:
function display_errors(alphas, split; content = nothing)
    alpha_padding = maximum(length.(alphas))
    number_padding = 16

    header_1 = rpad("Alpha", alpha_padding)
    header_2 = rpad("Avg Abs Error", number_padding)
    header_3 = rpad("Avg Rel Error", number_padding)
    header_4 = rpad("Max Error", number_padding)
    @info "$header_1 $header_2 $header_3 $header_4"

    for alpha in alphas
        avg_abs_error =
            rpad(average_abs_error(alpha, split; content = content), number_padding)
        avg_rel_error =
            rpad(average_rel_error(alpha, split; content = content), number_padding)
        max_error = rpad(maximum_error(alpha, split; content = content), number_padding)
        alpha = rpad(alpha, alpha_padding)
        @info "$alpha $avg_abs_error $avg_rel_error $max_error"
    end
end;

In [ ]:
potential_alphas = vcat(
    [
        ["$t/" * x for x in readdir(recommendee_alpha_basepath() * "/$t")] for
        t in ["all", task]
    ]...,
    ["$task/Transformer/explicit", "$task/Transformer/implicit"],
)
alphas = sort([
    x for
    x in potential_alphas if ispath("$(recommendee_alpha_basepath())/$(x)/alpha.jld2")
])
validation_alphas = sort(
    [
        ["$task/Linear$x" for x in ["Explicit", "Implicit"]]
        ["$task/Nonlinear$x" for x in ["Explicit", "Implicit"]]
        ["$task/Explicit"]
    ],
)
test_alphas = sort(["$task/MLE.Ensemble"])
inference_only_alphas = sort(["all/RecapSeries", "all/RelatedSeries"])
training_alphas = [
    x for
    x in alphas if x ∉ validation_alphas && x ∉ test_alphas && x ∉ inference_only_alphas
];

# Regular splits

In [ ]:
display_errors(training_alphas, "training")

In [ ]:
display_errors(training_alphas, "validation")
println()
display_errors(validation_alphas, "validation")

In [ ]:
display_errors(training_alphas, "test")
println()
display_errors(validation_alphas, "test")
println()
# display_errors(test_alphas, "test")

# Negative splits

In [ ]:
display_errors(training_alphas, "training"; content = "negative")

In [ ]:
display_errors(training_alphas, "validation"; content = "negative")
println()
display_errors(validation_alphas, "validation"; content = "negative")

In [ ]:
display_errors(training_alphas, "test"; content = "negative")
println()
display_errors(validation_alphas, "test"; content = "negative")
println()
# display_errors(test_alphas, "test"; content = "negative")